In [1]:
import gc
import os
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

/Users/shuozhang/anaconda/envs/py3k/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ["int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [3]:
def read_data():

    print("Reading files...")

    calendar = pd.read_csv(f"calendar.csv").pipe(reduce_mem_usage)
    sell_prices = pd.read_csv(f"sell_prices.csv").pipe(reduce_mem_usage)

    sales_train_val = pd.read_csv(f"sales_train_validation.csv",).pipe(
        reduce_mem_usage
    )
    submission = pd.read_csv(f"sample_submission.csv").pipe(
        reduce_mem_usage
    )

    print("calendar shape:", calendar.shape)
    print("sell_prices shape:", sell_prices.shape)
    print("sales_train_val shape:", sales_train_val.shape)
    print("submission shape:", submission.shape)

    return calendar, sell_prices, sales_train_val, submission

In [4]:
calendar, sell_prices, sales_train_val, submission = read_data()

NUM_ITEMS = sales_train_val.shape[0]  # 30490
DAYS_PRED = submission.shape[1] - 1

Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Mem. usage decreased to 95.00 Mb (78.7% reduction)
Mem. usage decreased to  2.09 Mb (84.5% reduction)
calendar shape: (1969, 14)
sell_prices shape: (6841121, 4)
sales_train_val shape: (30490, 1919)
submission shape: (60980, 29)


In [5]:
def encode_categorical(df, cols):
    for col in cols:
        # Leave NaN as it is.
        le = LabelEncoder()
        not_null = df[col][df[col].notnull()]
        df[col] = pd.Series(le.fit_transform(not_null), index=not_null.index)

    return df

In [6]:
calendar = encode_categorical(
    calendar, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
).pipe(reduce_mem_usage)

sales_train_val = encode_categorical(
    sales_train_val, ["item_id", "dept_id", "cat_id", "store_id", "state_id"],
).pipe(reduce_mem_usage)

sell_prices = encode_categorical(sell_prices, ["item_id", "store_id"]).pipe(
    reduce_mem_usage
)

Mem. usage decreased to  0.08 Mb (36.9% reduction)
Mem. usage decreased to 94.01 Mb (1.0% reduction)
Mem. usage decreased to 45.67 Mb (65.0% reduction)


In [7]:
import IPython
def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

In [8]:
def melt(sales_train_val, submission, nrows=55_000_000, verbose=True):
    # melt sales data, get it ready for training
    id_columns = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]

    # get product table.
    product = sales_train_val[id_columns]

    sales_train_val = pd.melt(sales_train_val,id_vars=id_columns, var_name="d", value_name="demand")

    sales_train_val = reduce_mem_usage(sales_train_val, verbose=False)

    if verbose:
        print("melted")
        display(sales_train_val)

    # separate test dataframes.
    vals = submission[submission["id"].str.endswith("validation")]
    evals = submission[submission["id"].str.endswith("evaluation")]

    # change column names.
    vals.columns = ["id"] + [f"d_{d}" for d in range(1914, 1914 + DAYS_PRED)]
    evals.columns = ["id"] + [f"d_{d}" for d in range(1942, 1942 + DAYS_PRED)]

    # merge with product table
    evals["id"] = evals["id"].str.replace("_evaluation", "_validation")
    vals = vals.merge(product, how="left", on="id")
    evals = evals.merge(product, how="left", on="id")
    evals["id"] = evals["id"].str.replace("_validation", "_evaluation")

    if verbose:
        print("validation")
        display(vals)

        print("evaluation")
        display(evals)

    vals = pd.melt(vals,id_vars=id_columns, var_name="d", value_name="demand")
    evals = pd.melt(evals, id_vars=id_columns, var_name="d", value_name="demand")

    sales_train_val["part"] = "train"
    vals["part"] = "validation"
    evals["part"] = "evaluation"

    data = pd.concat([sales_train_val, vals, evals], axis=0)

    del sales_train_val, vals, evals

    data = data.loc[nrows:]

    # delete evaluation for now.
    data = data[data["part"] != "evaluation"]

    gc.collect()

    if verbose:
        print("data")
        display(data)

    return data


def extract_d(df):
    return df["d"].str.extract(r"d_(\d+)").astype(np.int16)


def merge_calendar(data, calendar):
    calendar = calendar.drop(["weekday", "wday", "month", "year"], axis=1)
    return data.merge(calendar, how="left", on="d").assign(d=extract_d)


def merge_sell_prices(data, sell_prices):
    return data.merge(sell_prices, how="left", on=["store_id", "item_id", "wm_yr_wk"])

In [9]:
data = melt(sales_train_val, submission, nrows=27_500_000)
del sales_train_val
gc.collect()

melted


,id,item_id,dept_id,cat_id,store_id,state_id,d,demand
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,d_1,0
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,d_1,0
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,d_1,0
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,d_1,0
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,d_1,0


validation


,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1437,3,1,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1438,3,1,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1439,3,1,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1440,3,1,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1441,3,1,0,0


evaluation


,id,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,d_1951,d_1952,d_1953,d_1954,d_1955,d_1956,d_1957,d_1958,d_1959,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1437,3,1,0,0
1,HOBBIES_1_002_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1438,3,1,0,0
2,HOBBIES_1_003_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1439,3,1,0,0
3,HOBBIES_1_004_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1440,3,1,0,0
4,HOBBIES_1_005_CA_1_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1441,3,1,0,0


data


,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part
27500000,HOUSEHOLD_1_514_WI_3_validation,2506,5,2,9,2,d_902,16,train
27500001,HOUSEHOLD_1_515_WI_3_validation,2507,5,2,9,2,d_902,1,train
27500002,HOUSEHOLD_1_516_WI_3_validation,2508,5,2,9,2,d_902,1,train
27500003,HOUSEHOLD_1_517_WI_3_validation,2509,5,2,9,2,d_902,1,train
27500004,HOUSEHOLD_1_518_WI_3_validation,2510,5,2,9,2,d_902,0,train


252

In [10]:
data.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part
853715,FOODS_3_823_WI_3_validation,1432,2,0,9,2,d_1941,0,validation
853716,FOODS_3_824_WI_3_validation,1433,2,0,9,2,d_1941,0,validation
853717,FOODS_3_825_WI_3_validation,1434,2,0,9,2,d_1941,0,validation
853718,FOODS_3_826_WI_3_validation,1435,2,0,9,2,d_1941,0,validation
853719,FOODS_3_827_WI_3_validation,1436,2,0,9,2,d_1941,0,validation


In [11]:
train= data[data["part"].str.endswith("train")]

In [12]:
train.shape

(30827370, 9)

In [13]:
validation=data[data["part"].str.endswith("validation")]

In [14]:
validation.shape

(853720, 9)

In [15]:
data.shape

(31681090, 9)

In [16]:
data = merge_calendar(data, calendar)
del calendar
gc.collect()

267

In [17]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOUSEHOLD_1_514_WI_3_validation,2506,5,2,9,2,902,16,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0
1,HOUSEHOLD_1_515_WI_3_validation,2507,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0
2,HOUSEHOLD_1_516_WI_3_validation,2508,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0
3,HOUSEHOLD_1_517_WI_3_validation,2509,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0
4,HOUSEHOLD_1_518_WI_3_validation,2510,5,2,9,2,902,0,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0


In [18]:
data = merge_sell_prices(data, sell_prices)
del sell_prices
gc.collect()

70

In [19]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOUSEHOLD_1_514_WI_3_validation,2506,5,2,9,2,902,16,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,3.500000
1,HOUSEHOLD_1_515_WI_3_validation,2507,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,2.470703
2,HOUSEHOLD_1_516_WI_3_validation,2508,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,1.980469
3,HOUSEHOLD_1_517_WI_3_validation,2509,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,0.979980
4,HOUSEHOLD_1_518_WI_3_validation,2510,5,2,9,2,902,0,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,NaN


In [20]:
data = reduce_mem_usage(data)

Mem. usage decreased to 1722.17 Mb (0.0% reduction)


In [21]:
def add_demand_features(df):
    for diff in [0, 1, 2]:
        shift = DAYS_PRED + diff
        df[f"shift_t{shift}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(shift)
        )

    for size in [7, 30, 60, 90, 180]:
        df[f"rolling_std_t{size}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(DAYS_PRED).rolling(size).std()
        )

    for size in [7, 30, 60, 90, 180]:
        df[f"rolling_mean_t{size}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(DAYS_PRED).rolling(size).mean()
        )

    df["rolling_skew_t30"] = df.groupby(["id"])["demand"].transform(
        lambda x: x.shift(DAYS_PRED).rolling(30).skew()
    )
    df["rolling_kurt_t30"] = df.groupby(["id"])["demand"].transform(
        lambda x: x.shift(DAYS_PRED).rolling(30).kurt()
    )
    return df

In [22]:
def add_price_features(df):
    df["shift_price_t1"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.shift(1)
    )
    df["price_change_t1"] = (df["shift_price_t1"] - df["sell_price"]) / (
        df["shift_price_t1"]
    )
    df["rolling_price_max_t365"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.shift(1).rolling(365).max()
    )
    df["price_change_t365"] = (df["rolling_price_max_t365"] - df["sell_price"]) / (
        df["rolling_price_max_t365"]
    )

    df["rolling_price_std_t7"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.rolling(7).std()
    )
    df["rolling_price_std_t30"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.rolling(30).std()
    )
    return df.drop(["rolling_price_max_t365", "shift_price_t1"], axis=1)

In [23]:
def add_time_features(df, dt_col):
    df[dt_col] = pd.to_datetime(df[dt_col])
    attrs = [
        "year",
        "quarter",
        "month",
        "week",
        "day",
        "dayofweek",
        "is_year_end",
        "is_year_start",
        "is_quarter_end",
        "is_quarter_start",
        "is_month_end",
        "is_month_start",
    ]

    for attr in attrs:
        dtype = np.int16 if attr == "year" else np.int8
        df[attr] = getattr(df[dt_col].dt, attr).astype(dtype)

    df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(np.int8)
    return df

In [24]:
data = add_demand_features(data).pipe(reduce_mem_usage)

Mem. usage decreased to 2628.57 Mb (50.8% reduction)


In [25]:
data.head()
#shift 28: 28 days ago, no data
# rolling: before 7 days, no data

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,shift_t28,shift_t29,shift_t30,rolling_std_t7,rolling_std_t30,rolling_std_t60,rolling_std_t90,rolling_std_t180,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_skew_t30,rolling_kurt_t30
0,HOUSEHOLD_1_514_WI_3_validation,2506,5,2,9,2,902,16,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOUSEHOLD_1_515_WI_3_validation,2507,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,2.470703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOUSEHOLD_1_516_WI_3_validation,2508,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,1.980469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOUSEHOLD_1_517_WI_3_validation,2509,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,0.979980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOUSEHOLD_1_518_WI_3_validation,2510,5,2,9,2,902,0,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
data.tail(29)
# shift: 多少天前的数据
# rolling mean： 多少天前的平均值
# rolling std： 多少天前的标准方差
# rolling skew：判断多少天前是不是normal distribution
#For normally distributed data, the skewness should be about 0. 
#A skewness value > 0 means that there is more weight in the left tail of the distribution. 
#The function skewtest can be used to determine if the skewness value is close enough to 0, statistically speaking.
# rolling kurt：判断多少天前是不是normal distribution
#  return unbiased kurtosis over requested axis using Fisher’s definition of kurtosis (kurtosis of normal == 0.0）

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,shift_t28,shift_t29,shift_t30,rolling_std_t7,rolling_std_t30,rolling_std_t60,rolling_std_t90,rolling_std_t180,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_skew_t30,rolling_kurt_t30
31681061,FOODS_3_799_WI_3_validation,1408,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.179688,0.0,1.0,0.0,0.377930,0.614746,0.555176,0.541016,0.433594,0.142822,0.366699,0.283447,0.277832,0.172241,1.502930,1.332031
31681062,FOODS_3_800_WI_3_validation,1409,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.879883,10.0,6.0,8.0,2.572266,4.257812,5.203125,5.308594,6.574219,7.429688,8.929688,9.750000,10.710938,6.867188,0.213013,-0.656738
31681063,FOODS_3_801_WI_3_validation,1410,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.000000,2.0,7.0,3.0,2.339844,1.844727,2.558594,2.517578,3.732422,2.857422,2.333984,2.599609,2.722656,3.283203,1.166992,0.398926
31681064,FOODS_3_802_WI_3_validation,1411,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.679688,0.0,1.0,0.0,0.377930,0.678711,0.674805,0.625977,0.637207,0.142822,0.433350,0.449951,0.366699,0.344482,2.029297,5.667969
31681065,FOODS_3_803_WI_3_validation,1412,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.000000,0.0,0.0,0.0,0.000000,0.944336,1.725586,1.480469,1.273438,0.000000,0.266602,0.933105,0.789062,0.694336,4.679688,23.484375
31681066,FOODS_3_804_WI_3_validation,1413,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.879883,2.0,4.0,0.0,4.714844,5.375000,6.429688,6.078125,6.585938,4.714844,6.500000,7.867188,7.445312,9.359375,0.883301,0.392578
31681067,FOODS_3_805_WI_3_validation,1414,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.000000,1.0,5.0,0.0,2.000000,1.416016,1.462891,1.736328,2.232422,2.000000,1.833008,1.783203,2.078125,2.271484,0.393066,-0.707031
31681068,FOODS_3_806_WI_3_validation,1415,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.560547,1.0,0.0,0.0,1.112305,1.047852,0.988281,0.984863,0.988770,0.714355,0.933105,0.649902,0.544434,0.605469,0.717773,-0.768555
31681069,FOODS_3_807_WI_3_validation,1416,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.980469,0.0,3.0,0.0,3.644531,3.349609,3.003906,2.714844,2.269531,2.572266,4.500000,3.599609,3.099609,2.695312,0.769531,0.769531
31681070,FOODS_3_808_WI_3_validation,1417,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,0.939941,0.0,0.0,0.0,0.000002,8.554688,10.835938,11.531250,11.570312,0.000000,3.367188,9.914062,13.507812,8.625000,3.365234,12.968750


In [27]:
data = add_price_features(data).pipe(reduce_mem_usage)

Mem. usage decreased to 2870.28 Mb (0.0% reduction)


In [28]:
data.head(20)
# 如果有nan，计算都是nan
# shift change： 跟一天前价格变化多少
# max： 365天前最高价格跟今天价格变化
# rolling std：多少天前价格std

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,shift_t28,shift_t29,shift_t30,rolling_std_t7,rolling_std_t30,rolling_std_t60,rolling_std_t90,rolling_std_t180,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_skew_t30,rolling_kurt_t30,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30
0,HOUSEHOLD_1_514_WI_3_validation,2506,5,2,9,2,902,16,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOUSEHOLD_1_515_WI_3_validation,2507,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,2.470703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOUSEHOLD_1_516_WI_3_validation,2508,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,1.980469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOUSEHOLD_1_517_WI_3_validation,2509,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,0.979980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOUSEHOLD_1_518_WI_3_validation,2510,5,2,9,2,902,0,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,HOUSEHOLD_1_519_WI_3_validation,2511,5,2,9,2,902,0,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,4.878906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,HOUSEHOLD_1_520_WI_3_validation,2512,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,5.480469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,HOUSEHOLD_1_521_WI_3_validation,2513,5,2,9,2,902,4,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,0.970215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,HOUSEHOLD_1_522_WI_3_validation,2514,5,2,9,2,902,0,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,1.839844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,HOUSEHOLD_1_523_WI_3_validation,2515,5,2,9,2,902,0,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
data.tail(20)

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,shift_t28,shift_t29,shift_t30,rolling_std_t7,rolling_std_t30,rolling_std_t60,rolling_std_t90,rolling_std_t180,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_skew_t30,rolling_kurt_t30,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30
31681070,FOODS_3_808_WI_3_validation,1417,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,0.939941,0.0,0.0,0.0,0.000002,8.554688,10.835938,11.531250,11.570312,0.000000,3.367188,9.914062,13.507812,8.625000,3.365234,12.968750,0.0,0.000000,0.0,0.000000e+00
31681071,FOODS_3_809_WI_3_validation,1418,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.000000,0.0,1.0,6.0,2.339844,2.279297,2.023438,2.177734,4.089844,1.857422,2.099609,2.150391,2.410156,3.355469,0.859863,-0.214966,0.0,0.000000,0.0,0.000000e+00
31681072,FOODS_3_810_WI_3_validation,1419,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.560547,1.0,1.0,1.0,0.689941,1.332031,1.197266,1.030273,0.816406,0.856934,0.866699,0.700195,0.466553,0.355469,2.322266,6.761719,0.0,0.000000,0.0,0.000000e+00
31681073,FOODS_3_811_WI_3_validation,1420,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.879883,28.0,10.0,24.0,7.933594,9.500000,9.007812,8.562500,10.156250,21.578125,18.406250,21.640625,22.093750,17.328125,0.326904,-0.937500,0.0,0.000000,0.0,0.000000e+00
31681074,FOODS_3_812_WI_3_validation,1421,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.939453,3.0,2.0,0.0,1.214844,1.442383,1.445312,1.348633,1.220703,0.856934,1.700195,1.250000,1.188477,1.133789,1.233398,2.023438,0.0,0.000000,0.0,0.000000e+00
31681075,FOODS_3_813_WI_3_validation,1422,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,4.539062,0.0,1.0,0.0,0.786621,0.503906,0.473877,0.444092,0.469238,0.571289,0.233276,0.250000,0.222168,0.205566,2.154297,4.250000,0.0,0.000000,0.0,0.000000e+00
31681076,FOODS_3_814_WI_3_validation,1423,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,1.879883,4.0,6.0,1.0,2.160156,1.858398,2.060547,2.005859,1.897461,2.000000,2.166016,2.232422,2.445312,2.099609,0.637695,-0.654785,0.0,0.000000,0.0,0.000000e+00
31681077,FOODS_3_815_WI_3_validation,1424,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.480469,0.0,0.0,0.0,0.377930,0.305176,0.444336,0.517090,0.532715,0.142822,0.099976,0.150024,0.188843,0.199951,2.808594,6.308594,0.0,0.000000,0.0,0.000000e+00
31681078,FOODS_3_816_WI_3_validation,1425,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,0.250000,0.0,0.0,0.0,0.377930,4.101562,8.796875,10.320312,10.289062,0.142822,3.833984,8.398438,10.835938,12.320312,0.795410,-0.763672,0.0,0.000000,0.0,0.000000e+00
31681079,FOODS_3_817_WI_3_validation,1426,2,0,9,2,1941,0,validation,2016-05-22,11617,NaN,NaN,NaN,NaN,0,0,0,2.939453,1.0,0.0,0.0,0.488037,0.406738,0.508301,0.553711,0.499756,0.285645,0.199951,0.250000,0.311035,0.238892,1.580078,0.527344,0.0,0.000000,0.0,0.000000e+00


In [30]:
data = add_time_features(data, dt_col).pipe(reduce_mem_usage)

Mem. usage decreased to 3293.27 Mb (0.0% reduction)


In [31]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,shift_t28,shift_t29,shift_t30,rolling_std_t7,rolling_std_t30,rolling_std_t60,rolling_std_t90,rolling_std_t180,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_skew_t30,rolling_kurt_t30,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,year,quarter,month,week,day,dayofweek,is_year_end,is_year_start,is_quarter_end,is_quarter_start,is_month_end,is_month_start,is_weekend
0,HOUSEHOLD_1_514_WI_3_validation,2506,5,2,9,2,902,16,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,3,7,29,18,3,0,0,0,0,0,0,0
1,HOUSEHOLD_1_515_WI_3_validation,2507,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,2.470703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,3,7,29,18,3,0,0,0,0,0,0,0
2,HOUSEHOLD_1_516_WI_3_validation,2508,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,1.980469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,3,7,29,18,3,0,0,0,0,0,0,0
3,HOUSEHOLD_1_517_WI_3_validation,2509,5,2,9,2,902,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,0.979980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,3,7,29,18,3,0,0,0,0,0,0,0
4,HOUSEHOLD_1_518_WI_3_validation,2510,5,2,9,2,902,0,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,3,7,29,18,3,0,0,0,0,0,0,0


In [32]:
data = data.sort_values("date")

In [33]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,shift_t28,shift_t29,shift_t30,rolling_std_t7,rolling_std_t30,rolling_std_t60,rolling_std_t90,rolling_std_t180,rolling_mean_t7,rolling_mean_t30,rolling_mean_t60,rolling_mean_t90,rolling_mean_t180,rolling_skew_t30,rolling_kurt_t30,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,year,quarter,month,week,day,dayofweek,is_year_end,is_year_start,is_quarter_end,is_quarter_start,is_month_end,is_month_start,is_weekend
0,HOUSEHOLD_1_514_WI_3_validation,2506,5,2,9,2,902,16,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,3,7,29,18,3,0,0,0,0,0,0,0
1327,FOODS_3_172_WI_3_validation,784,2,0,9,2,902,0,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,3,7,29,18,3,0,0,0,0,0,0,0
1326,FOODS_3_171_WI_3_validation,783,2,0,9,2,902,0,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,3,7,29,18,3,0,0,0,0,0,0,0
1325,FOODS_3_170_WI_3_validation,782,2,0,9,2,902,0,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,3,7,29,18,3,0,0,0,0,0,0,0
1324,FOODS_3_169_WI_3_validation,781,2,0,9,2,902,0,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,3.980469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,3,7,29,18,3,0,0,0,0,0,0,0


In [34]:
print("start date:", data[dt_col].min())
print("end date:", data[dt_col].max())
print("data shape:", data.shape)

start date: 2013-07-18 00:00:00
end date: 2016-05-22 00:00:00
data shape: (31681090, 51)


In [35]:
data.to_csv('data1.csv',index=False)

In [ ]:
# next: filter demand ?
# feature selection？
# add holiday？
# split strategies: cross validation by time? folder？
# model: lightgbm, xgboost, catboost, cnn, time series 
# evaluation metric: rmse, wrmsse?